## Clone the official repo of style-GAN from Nvidia

In [5]:
! git clone https://github.com/NVlabs/stylegan

Cloning into 'stylegan'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [6]:
# Go to the directory with model
%cd stylegan/

/content/stylegan/stylegan


## Load necessary library and the pretrained model

Note that the original code was in tensorflow 1.x version so you need to make sure you use the right version (not 2.x)

In [0]:
%tensorflow_version 1.x

In [8]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
!pip install ipython-autotime
!nvidia-smi

#----------------------------------------------------------------------------
# Helpers for loading and using pre-trained generators.

url_ffhq        = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
url_celebahq    = 'https://drive.google.com/uc?id=1MGqJl28pN4t7SAtSrPdSRJSQJqahkzUf' # karras2019stylegan-celebahq-1024x1024.pkl
url_bedrooms    = 'https://drive.google.com/uc?id=1MOSKeGF0FJcivpBI7s63V9YHloUTORiF' # karras2019stylegan-bedrooms-256x256.pkl
url_cars        = 'https://drive.google.com/uc?id=1MJ6iCfNtMIRicihwRorsM3b7mmtmK9c3' # karras2019stylegan-cars-512x384.pkl
url_cats        = 'https://drive.google.com/uc?id=1MQywl0FNt6lHu8E_EUqnRbviagS7fbiJ' # karras2019stylegan-cats-256x256.pkl

synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)
_Gs_cache = dict()
def load_Gs(url):
    if url not in _Gs_cache:
        with dnnlib.util.open_url(url, num_attempts=10, cache_dir=os.path.join(os.getcwd(), "cache_dir")) as f:
            _G, _D, Gs = pickle.load(f)
        _Gs_cache[url] = Gs
    return _Gs_cache[url]

# Load the pretrained model for face
Gs = load_Gs(url_ffhq)


NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



OSError: ignored

In [0]:
# Mount google drive to download the images (if you use Google cloud service)
from google.colab import drive
drive.mount('/content/drive/')
root_dir = '/content/drive/'

## Show example mixing images

In [0]:
# Style mixing - mix n_mix_layer of mix n_mix_layer of mixing image to reference image.
from itertools import chain

tflib.init_tf()
result_dir = os.path.join(root_dir, "result_dir")
os.makedirs(result_dir, exist_ok=True)
example_file_name = os.path.join(result_dir, 'example-style-mixing.png')

w = 1024
h = 1024
style_ranges = [range(8, 18), range(2, 4), range(2, 4), range(0, 4)] 
master_seeds = [701,687,615,2268,888,829,1898,1733,1614,845,1967,1555,1157,1012,91,388,201,500]
src_seeds = [701,687,615,2268,888,829,1898,1733,1614,1967,1555,1157,1012,91,388,201,500] # mixing images
dst_seeds = [master_seeds[9]] * len(style_ranges) # original image

src_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in src_seeds)
dst_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in dst_seeds)
src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)

canvas = PIL.Image.new('RGB', (w * (len(src_seeds) + 1), h * (len(dst_seeds) + 1)), 'white')
for col, src_image in enumerate(list(src_images)):
    canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
for row, dst_image in enumerate(list(dst_images)):
    canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))
    row_dlatents = np.stack([dst_dlatents[row]] * len(src_seeds))
    row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
    row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
    for col, image in enumerate(list(row_images)):
        canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h))
canvas.save(example_file_name)

## Create the set of mixing images and save

In [0]:
# Style mixing - mix n_mix_layer of mix n_mix_layer of mixing image to reference image.
from itertools import chain
import timeit
start = timeit.timeit()

w = 1024
h = 1024
style_ranges = [range(8,18), range(2,4)]
master_seeds = [701,687,615,2268,888,829,1898,1733,1614,845,1967,1555,1157,1012,91,388,201,500]
mix_seeds = master_seeds 
original_seeds = master_seeds

mix_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in mix_seeds)
original_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in original_seeds)
mix_dlatents = Gs.components.mapping.run(mix_latents, None) # [seed, layer, component]
original_dlatents = Gs.components.mapping.run(original_latents, None) # [seed, layer, component]
mix_images = Gs.components.synthesis.run(mix_dlatents, randomize_noise=False, **synthesis_kwargs)
original_images = Gs.components.synthesis.run(original_dlatents, randomize_noise=False, **synthesis_kwargs)

for original_ind, original_image in enumerate(list(original_images)):
    # Save original image
    canvas_original = PIL.Image.new(mode = 'RGB', size = (w, h), color = 'white')
    canvas_original.paste(PIL.Image.fromarray(original_image, 'RGB'), (0, 0))
    file_name = "{}_original.png".format(original_seeds[original_ind])
    canvas_original.save(os.path.join(result_dir, file_name))

    for style_ind, style in enumerate(list(style_ranges)):
        # Mix the images
        row_dlatents = np.stack([original_dlatents[original_ind]] * len(mix_seeds))
        row_dlatents[:, style] = mix_dlatents[:, style]
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)

        # Save the mix images
        for variant, image in enumerate(list(row_images)):
            canvas_mix = PIL.Image.new('RGB', (w, h), 'white')
            canvas_mix.paste(PIL.Image.fromarray(image, 'RGB'), (0, 0))
            file_name = "{}_{}_{}.png".format(original_seeds[original_ind], style, mix_seeds[variant])
            canvas_mix.save(os.path.join(result_dir, file_name))

end = timeit.timeit()
print(end - start)